In [449]:
import pandas as pd
import plotly.express as px
import kaleido

df = pd.read_csv(
    "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/active_cases_2020-07-17_0800.csv"
)

state_area = pd.read_csv("./agricultural_data.csv")
# Drop the unnecessary columns
state_area = state_area.drop([" Kharif", " Rabi"], axis=1)
# Rename the remaining column
state_area = state_area.rename(columns={" Summer": "area"})


reModel2 = pd.read_csv("./##Model2_reallocated.csv")
BenifitModel2 = pd.read_csv("./#Model2_rotationbenefit.csv")
reModel1 = pd.read_csv("./##Model1_reallocated.csv")

In [450]:
mapping = {
    "odisha": "Odisha",
    "jharkhand": "Jharkhand",
    "manipur": "Manipur",
    "nagaland": "Nagaland",
    "madhya_pradesh": "Madhya Pradesh",
    "uttarakhand": "Uttarakhand",
    "uttar_pradesh": "Uttar Pradesh",
    "mizoram": "Mizoram",
    "meghalaya": "Meghalaya",
    "maharashtra": "Maharashtra",
    "west_bengal": "West Bengal",
    "gujarat": "Gujarat",
    "assam": "Assam",
    "bihar": "Bihar",
    "jammu_and_kashmir": "Jammu & Kashmir",
    "dadra_and_nagar_haveli": "Dadra and Nagar Haveli and Daman and Diu",
    "arunachal_pradesh": "Arunachal Pradesh",
    "karnataka": "Karnataka",
    "chhattisgarh": "Chhattisgarh",
    "tripura": "Tripura",
    "himachal_pradesh": "Himachal Pradesh",
    "chandigarh": "Chandigarh",
    "telangana": "Telangana",
    "sikkim": "Sikkim",
    "rajasthan": "Rajasthan",
    "punjab": "Punjab",
    "andhra_pradesh": "Andhra Pradesh",
    "haryana": "Haryana",
    "kerala": "Kerala",
    "tamil_nadu": "Tamil Nadu",
    "puducherry": "Puducherry",
    "andaman_and_nicobar_islands": "Andaman & Nicobar",
    "goa": "Goa",
}

In [451]:
state_area['state']

0                            Andaman & Nicobar
1                               Andhra Pradesh
2                            Arunachal Pradesh
3                                        Assam
4                                        Bihar
5                                   Chandigarh
6                                 Chhattisgarh
7     Dadra and Nagar Haveli and Daman and Diu
8                                          Goa
9                                      Gujarat
10                                     Haryana
11                            Himachal Pradesh
12                             Jammu & Kashmir
13                                   Jharkhand
14                                   Karnataka
15                                      Kerala
16                              Madhya Pradesh
17                                 Maharashtra
18                                     Manipur
19                                   Meghalaya
20                                     Mizoram
21           

In [454]:
BenifitModel2["Rabi"] = pd.to_numeric(BenifitModel2["Rabi"], errors="coerce")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   crop    24 non-null     object 
 1   state   24 non-null     object 
 2   Rabi    24 non-null     object 
 3   Summer  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 900.0+ bytes


In [453]:
import pandas as pd


def modify_state(df, mapping):
    df["state"] = df["state"].str.replace("^.", "", regex=True)
    df["state"] = df["state"].replace(mapping)
    return df


def divide_columns(df, state_area):
    df = pd.merge(df, state_area, on="state", how="left")
    for column in ["Kharif", "Rabi", "Summer"]:
        df[column] = (df[column] / 1000).round(0)
    return df


def divide_columns2(df, state_area):
    df = pd.merge(df, state_area, on="state", how="left")
    for column in ["Rabi"]:
        df[column] = (df[column] / 1000).round(0)
    return df


# Modify the 'state' column and merge with state_area
reModel1 = modify_state(reModel1, mapping)
reModel2 = modify_state(reModel2, mapping)
BenifitModel2 = modify_state(BenifitModel2, mapping)

# Divide the 'Kharif', 'Rabi', 'Summer' columns by the 'area' column
reModel1 = divide_columns(reModel1, state_area)
reModel2 = divide_columns(reModel2, state_area)
BenifitModel2 = divide_columns2(BenifitModel2, state_area)

print(reModel1.head(), "\n\n")
print(reModel2.head(), "\n\n")
print(BenifitModel2.head(), "\n\n")

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
BenifitModel2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   crop    24 non-null     object 
 1   state   24 non-null     object 
 2   Rabi    24 non-null     object 
 3   Summer  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 900.0+ bytes


In [ ]:
# Create a new 'text' column, formatted as 'crop: value'
BenifitModel2["text"] = BenifitModel2["crop"] + ": " + (BenifitModel2["Rabi"]).astype(str)+'k'

# Use groupby to combine all text information for the same state
grouped_text = (
    BenifitModel2.groupby("state")["text"].apply(lambda x: "<br>".join(x)).reset_index()
)

# Merge the combined text information back into the original DataFrame
BenifitModel2_combined = BenifitModel2.drop("text", axis=1).merge(
    grouped_text, on="state"
)

# Remove duplicate rows, keeping only a unique row for each state
BenifitModel2_combined = BenifitModel2_combined.drop_duplicates(
    subset="state"
).reset_index(drop=True)

# Display the first few rows of the combined DataFrame
BenifitModel2_combined

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
import pandas as pd

# 假设 reModel1 是您的原始 DataFrame

# 先将 reModel1 转换为 "长格式" (long format)
long_df = reModel1.melt(
    id_vars=["crop", "state", "area"],
    value_vars=["Kharif", "Rabi", "Summer"],
    var_name="season",
    value_name="value",
)

# 创建每个季节的数据集
m1_Kharif = long_df[(long_df["season"] == "Kharif") & (long_df["value"].notna())]
m1_Rabi = long_df[(long_df["season"] == "Rabi") & (long_df["value"].notna())]
m1_Summer = long_df[(long_df["season"] == "Summer") & (long_df["value"].notna())]

# 重置每个数据集的索引，并只保留需要的列
m1_Kharif = m1_Kharif[["state", "crop", "value"]].reset_index(drop=True)
m1_Rabi = m1_Rabi[["state", "crop", "value"]].reset_index(drop=True)
m1_Summer = m1_Summer[["state", "crop", "value"]].reset_index(drop=True)

# 显示每个数据集的前几行以进行检查
print(m1_Kharif.value_counts())
print(m1_Rabi.value_counts())
print(m1_Summer.value_counts())

state              crop      value 
Andaman & Nicobar  rice      4.0       1
Manipur            jute      68.0      1
Rajasthan          maize     3286.0    1
                   cotton    4669.0    1
Punjab             rice      1487.0    1
                                      ..
Himachal Pradesh   maize     176.0     1
                   cotton    124.0     1
Haryana            rice      1222.0    1
                   maize     1222.0    1
West Bengal        soyabean  1801.0    1
Name: count, Length: 113, dtype: int64
state              crop       value 
Andaman & Nicobar  maize      4.0       1
Andhra Pradesh     horsegram  2042.0    1
Odisha             potato     963.0     1
                   maize      963.0     1
                   horsegram  677.0     1
                                       ..
Jammu & Kashmir    potato     221.0     1
Himachal Pradesh   wheat      176.0     1
                   potato     176.0     1
Haryana            wheat      1222.0    1
West Bengal      

In [ ]:
import pandas as pd

# 假设 reModel1 是您的原始 DataFrame

# 先将 reModel1 转换为 "长格式" (long format)
long_df = reModel2.melt(
    id_vars=["crop", "state", "area"],
    value_vars=["Kharif", "Rabi", "Summer"],
    var_name="season",
    value_name="value",
)

# 创建每个季节的数据集
m2_Kharif = long_df[(long_df["season"] == "Kharif") & (long_df["value"].notna())]
m2_Rabi = long_df[(long_df["season"] == "Rabi") & (long_df["value"].notna())]


# 重置每个数据集的索引，并只保留需要的列
m2_Kharif = m2_Kharif[["state", "crop", "value"]].reset_index(drop=True)
m2_Rabi = m2_Rabi[["state", "crop", "value"]].reset_index(drop=True)

# 显示每个数据集的前几行以进行检查
print(m2_Rabi.value_counts())
print(m2_Kharif.value_counts())

state              crop       value 
Andaman & Nicobar  maize      4.0       1
Andhra Pradesh     horsegram  2042.0    1
Odisha             rice       963.0     1
                   potato     963.0     1
                   maize      963.0     1
                                       ..
Jammu & Kashmir    potato     221.0     1
Himachal Pradesh   wheat      176.0     1
                   potato     176.0     1
Haryana            wheat      1222.0    1
West Bengal        rice       1267.0    1
Name: count, Length: 80, dtype: int64
state              crop      value 
Andaman & Nicobar  rice      4.0       1
Manipur            jute      68.0      1
Rajasthan          maize     3286.0    1
                   cotton    4669.0    1
Punjab             rice      1487.0    1
                                      ..
Himachal Pradesh   potato    176.0     1
                   maize     124.0     1
Haryana            rice      1222.0    1
                   maize     1222.0    1
West Bengal      

In [ ]:
m2_Rabi

,state,crop,value
0,Andhra Pradesh,rice,2042.0
1,Assam,rice,776.0
2,Bihar,rice,1020.0
3,Dadra and Nagar Haveli and Daman and Diu,rice,4.0
4,Goa,rice,30.0
...,...,...,...
75,Uttar Pradesh,potato,4905.0
76,Uttarakhand,potato,210.0
77,West Bengal,potato,1801.0
78,Maharashtra,soyabean,4291.0


In [ ]:
import pandas as pd

# 假设 m1_Kharif 是您的 DataFrame

# 首先，创建一个新的 'text' 列，格式为 'crop: value'
m1_Kharif["text"] = m1_Kharif["crop"] + ": " + m1_Kharif["value"].astype(str) + "k"

# 使用 groupby 来合并同一个州的所有作物信息
grouped_text = (
    m1_Kharif.groupby("state")["text"].apply(lambda x: ", <br> ".join(x)).reset_index()
)

# 将合并后的文本信息重新合并到原始 DataFrame
m1_Kharif_combined = m1_Kharif.drop("text", axis=1).merge(grouped_text, on="state")

# 删除重复行，保留每个州的唯一行
m1_Kharif_combined = m1_Kharif_combined.drop_duplicates(subset="state").reset_index(
    drop=True
)
m1_Kharif_combined["text"] = m1_Kharif_combined["text"].apply(
    lambda x: "<br>".join(x.split("<br>")[:3])
)
# 显示合并后的 DataFrame 的前几行
m1_Kharif_combined.head()

,state,crop,value,text
0,Andaman & Nicobar,rice,4.0,rice: 4.0k
1,Andhra Pradesh,rice,2042.0,"rice: 2042.0k, <br> cotton: 2042.0k, <br> maiz..."
2,Arunachal Pradesh,rice,53.0,"rice: 53.0k, <br> maize: 53.0k, <br> soyabean:..."
3,Assam,rice,776.0,"rice: 776.0k, <br> cotton: 776.0k, <br> maize:..."
4,Chandigarh,rice,0.0,"rice: 0.0k, <br> maize: 0.0k"


In [ ]:
import pandas as pd

# Assuming m1_Rabi is your DataFrame

# First, create a new 'text' column, formatted as 'crop: value'
m1_Rabi["text"] = m1_Rabi["crop"] + ": " + m1_Rabi["value"].astype(str)+'k'

# Use groupby to combine all crop information for the same state
grouped_text = (
    m1_Rabi.groupby("state")["text"].apply(lambda x: ", <br> ".join(x)).reset_index()
)

# Merge the combined text information back into the original DataFrame
m1_Rabi_combined = m1_Rabi.drop("text", axis=1).merge(grouped_text, on="state")

# Remove duplicate rows, keeping only a unique row for each state
m1_Rabi_combined = m1_Rabi_combined.drop_duplicates(subset="state").reset_index(
    drop=True
)
m1_Rabi_combined["text"] = m1_Rabi_combined["text"].apply(
    lambda x: "<br>".join(x.split("<br>")[:3])
)
# Display the first few rows of the combined DataFrame
m1_Rabi_combined.head()

,state,crop,value,text
0,Andhra Pradesh,rice,2042.0,"rice: 2042.0k, <br> wheat: 1437.0k, <br> maize..."
1,Assam,rice,776.0,"rice: 776.0k, <br> wheat: 776.0k"
2,Bihar,rice,1020.0,"rice: 1020.0k, <br> wheat: 1450.0k, <br> maize..."
3,Dadra and Nagar Haveli and Daman and Diu,rice,4.0,"rice: 4.0k, <br> wheat: 4.0k, <br> maize: 4.0k"
4,Goa,rice,30.0,rice: 30.0k


In [ ]:
import pandas as pd

# Assuming m2_Rabi is your DataFrame

# First, create a new 'text' column, formatted as 'crop: value'
m2_Rabi["text"] = m2_Rabi["crop"] + ": " + m2_Rabi["value"].astype(str) + "k"

# Use groupby to combine all crop information for the same state
grouped_text = (
    m2_Rabi.groupby("state")["text"].apply(lambda x: ", <br> ".join(x)).reset_index()
)

# Merge the combined text information back into the original DataFrame
m2_Rabi_combined = m2_Rabi.drop("text", axis=1).merge(grouped_text, on="state")

# Remove duplicate rows, keeping only a unique row for each state
m2_Rabi_combined = m2_Rabi_combined.drop_duplicates(subset="state").reset_index(
    drop=True
)
m2_Rabi_combined["text"] = m2_Rabi_combined["text"].apply(
    lambda x: "<br>".join(x.split("<br>")[:3])
)
# Display the first few rows of the combined DataFrame
m2_Rabi_combined.head()

,state,crop,value,text
0,Andhra Pradesh,rice,2042.0,"rice: 2042.0k, <br> wheat: 1437.0k, <br> maize..."
1,Assam,rice,776.0,"rice: 776.0k, <br> wheat: 776.0k"
2,Bihar,rice,1020.0,"rice: 1020.0k, <br> wheat: 1450.0k, <br> maize..."
3,Dadra and Nagar Haveli and Daman and Diu,rice,4.0,"rice: 4.0k, <br> wheat: 4.0k, <br> maize: 4.0k"
4,Goa,rice,30.0,rice: 30.0k


In [ ]:
import pandas as pd

# Assuming m2_Kharif is your DataFrame

# First, create a new 'text' column, formatted as 'crop: value'
m2_Kharif["text"] = m2_Kharif["crop"] + ": " + m2_Kharif["value"].astype(str) + "k"

# Use groupby to combine all crop information for the same state
grouped_text = (
    m2_Kharif.groupby("state")["text"].apply(lambda x: ", <br> ".join(x)).reset_index()
)

# Merge the combined text information back into the original DataFrame
m2_Kharif_combined = m2_Kharif.drop("text", axis=1).merge(grouped_text, on="state")

# Remove duplicate rows, keeping only a unique row for each state
m2_Kharif_combined = m2_Kharif_combined.drop_duplicates(subset="state").reset_index(
    drop=True
)
m2_Kharif_combined["text"] = m2_Kharif_combined["text"].apply(
    lambda x: "<br>".join(x.split("<br>")[:3])
)
# Display the first few rows of the combined DataFrame
m2_Kharif_combined.head()

,state,crop,value,text
0,Andaman & Nicobar,rice,4.0,rice: 4.0k
1,Andhra Pradesh,rice,2042.0,"rice: 2042.0k, <br> cotton: 2042.0k, <br> maiz..."
2,Arunachal Pradesh,rice,53.0,"rice: 53.0k, <br> maize: 53.0k, <br> soyabean:..."
3,Assam,rice,776.0,"rice: 776.0k, <br> cotton: 776.0k, <br> maize:..."
4,Chandigarh,rice,0.0,"rice: 0.0k, <br> maize: 0.0k"


In [ ]:
BenifitModel2_combined

,crop,state,Rabi,Summer,text
0,wheat,Andhra Pradesh,1437297.75,NaN,wheat: 1437297.75
1,wheat,Arunachal Pradesh,52920,NaN,wheat: 52920
2,wheat,Assam,776459.25,NaN,wheat: 776459.25
3,wheat,Chandigarh,378,NaN,wheat: 378
4,wheat,Dadra and Nagar Haveli and Daman and Diu,4488.75,NaN,wheat: 4488.75
5,wheat,Gujarat,2407198.5,NaN,wheat: 2407198.5
6,wheat,Haryana,1222452,NaN,wheat: 1222452
7,wheat,Himachal Pradesh,175959,NaN,wheat: 175959
8,wheat,Jammu & Kashmir,221082.75,NaN,wheat: 221082.75
9,wheat,Jharkhand,298289.25,NaN,wheat: 298289.25


In [ ]:
# Merge the two DataFrames on 'state'
merged_df_Rabi = pd.merge(
    BenifitModel2_combined,
    m2_Rabi_combined[["state", "text"]],
    on="state",
    suffixes=("_Benifit", "_Rabi"),
)

# Combine the 'text' columns
merged_df_Rabi["text"] = (
    "Rotation Benifit:<br>"
    + merged_df_Rabi["text_Benifit"]
    + "<br>"
    + "Area Size:<br>"
    + merged_df_Rabi["text_Rabi"]
)

# Drop the original 'text' columns
merged_df_Rabi = merged_df_Rabi.drop(["text_Benifit", "text_Rabi"], axis=1)

# Display the first few rows of the merged DataFrame
merged_df_Rabi.head()

,crop,state,Rabi,Summer,text
0,wheat,Andhra Pradesh,1437297.75,NaN,Rotation Benifit:<br>wheat: 1437297.75<br>Area...
1,wheat,Arunachal Pradesh,52920,NaN,Rotation Benifit:<br>wheat: 52920<br>Area Size...
2,wheat,Assam,776459.25,NaN,Rotation Benifit:<br>wheat: 776459.25<br>Area ...
3,wheat,Chandigarh,378,NaN,Rotation Benifit:<br>wheat: 378<br>Area Size:<...
4,wheat,Dadra and Nagar Haveli and Daman and Diu,4488.75,NaN,Rotation Benifit:<br>wheat: 4488.75<br>Area Si...


In [ ]:
# Merge the two DataFrames on 'state'
merged_df_Kharif = pd.merge(
    BenifitModel2_combined,
    m2_Kharif_combined[["state", "text"]],
    on="state",
    suffixes=("_Benifit", "_Kharif"),
)

# Combine the 'text' columns
merged_df_Kharif["text"] = "Area Size:<br>" + merged_df_Kharif["text_Kharif"]

# Drop the original 'text' columns
merged_df_Kharif = merged_df_Kharif.drop(["text_Benifit", "text_Kharif"], axis=1)

# Fill NaN values in 'Summer' column with 0
merged_df_Kharif["Summer"] = merged_df_Kharif["Summer"].fillna(0)

# Display the first few rows of the DataFrame
merged_df_Kharif.head()

,crop,state,Rabi,Summer,text
0,wheat,Andhra Pradesh,1437297.75,0.0,"Area Size:<br>rice: 2042.0k, <br> cotton: 2042..."
1,wheat,Arunachal Pradesh,52920,0.0,"Area Size:<br>rice: 53.0k, <br> maize: 53.0k, ..."
2,wheat,Assam,776459.25,0.0,"Area Size:<br>rice: 776.0k, <br> cotton: 776.0..."
3,wheat,Chandigarh,378,0.0,"Area Size:<br>rice: 0.0k, <br> maize: 0.0k"
4,wheat,Dadra and Nagar Haveli and Daman and Diu,4488.75,0.0,"Area Size:<br>rice: 4.0k, <br> maize: 4.0k"


In [ ]:
import plotly.graph_objects as go
import geopandas as gpd
import kaleido

# 读取地理数据
geojson_url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
geo_df = gpd.read_file(geojson_url)

geo_df

# 使用 m1_Kharif 数据合并地理数据
geo_df = geo_df.merge(m1_Kharif_combined, left_on="ST_NM", right_on="state")

# 创建 Choropleth 图层
fig = go.Figure(
    data=go.Choropleth(
        geojson=geojson_url,
        featureidkey="properties.ST_NM",
        locations=geo_df["state"],
        z=geo_df["value"],  # 使用 'value' 作为区分的依据
        text=geo_df["text"],  # 使用合并的 'text' 列
        colorscale=["white", "white"],  # 更改色阶以更好地显示数据
        hoverinfo="location+text",
        marker_line_color="burlywood",
    )
)

# 计算每个州的中心点坐标
geo_df["center"] = geo_df["geometry"].centroid
geo_df["lon"] = geo_df["center"].apply(lambda p: p.x)
geo_df["lat"] = geo_df["center"].apply(lambda p: p.y)

# 添加散点图层以固定显示文本
fig.add_trace(
    go.Scattergeo(
        lon=geo_df["lon"],
        lat=geo_df["lat"],
        text=geo_df["text"],
        mode="text",
        textfont=dict(size=8, color="darkred"),  # 设置字体大小，您可以根据需要调整
    )
)

fig.update_geos(fitbounds="locations", visible=False)

fig.update_layout(
    width=1800,  # 宽度, 您可以根据需要调整
    height=1200,  # 高度, 您可以根据需要调整
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)
fig.show()
fig.write_image("#model1_Karif.png", scale=2)

C:\Users\redhawc\AppData\Local\Temp\ipykernel_55160\630832872.py:29: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [ ]:
import plotly.graph_objects as go
import geopandas as gpd
import kaleido

# 读取地理数据
geojson_url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
geo_df = gpd.read_file(geojson_url)

geo_df

# 使用 m1_Kharif 数据合并地理数据
geo_df = geo_df.merge(m1_Rabi_combined, left_on="ST_NM", right_on="state")

# 创建 Choropleth 图层
fig = go.Figure(
    data=go.Choropleth(
        geojson=geojson_url,
        featureidkey="properties.ST_NM",
        locations=geo_df["state"],
        z=geo_df["value"],  # 使用 'value' 作为区分的依据
        text=geo_df["text"],  # 使用合并的 'text' 列
        colorscale=["white", "white"],  # 更改色阶以更好地显示数据
        hoverinfo="location+text",
        marker_line_color="burlywood",
    )
)

# 计算每个州的中心点坐标
geo_df["center"] = geo_df["geometry"].centroid
geo_df["lon"] = geo_df["center"].apply(lambda p: p.x)
geo_df["lat"] = geo_df["center"].apply(lambda p: p.y)

# 添加散点图层以固定显示文本
fig.add_trace(
    go.Scattergeo(
        lon=geo_df["lon"],
        lat=geo_df["lat"],
        text=geo_df["text"],
        mode="text",
        textfont=dict(size=8, color="darkred"),  # 设置字体大小，您可以根据需要调整
    )
)

fig.update_geos(fitbounds="locations", visible=False)

fig.update_layout(
    width=1800,  # 宽度, 您可以根据需要调整
    height=1200,  # 高度, 您可以根据需要调整
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)
fig.show()
fig.write_image("#model1_Rabi.png", scale=2)

C:\Users\redhawc\AppData\Local\Temp\ipykernel_55160\1121761887.py:29: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [ ]:
import plotly.graph_objects as go
import geopandas as gpd

# ...（读取地理数据的代码）
geojson_url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
geo_df = gpd.read_file(geojson_url)
# 假设 BenifitModel2 是您的数据集
geo_df = geo_df.merge(merged_df_Kharif, left_on="ST_NM", right_on="state")

# 创建 Choropleth 图层
fig = go.Figure(
    data=go.Choropleth(
        geojson=geojson_url,
        featureidkey="properties.ST_NM",
        locations=geo_df["state"],
        z=geo_df["Summer"],  # 使用 'Rabi' 作为区分的依据
        text=geo_df["text"],  # 使用合并的 'text' 列
        colorscale=["white", "white"],  # 使用浅灰色到绿色的色阶
        hoverinfo="location+text",
        marker_line_color="burlywood",
    )
)

# 计算每个州的中心点坐标
geo_df["center"] = geo_df["geometry"].centroid
geo_df["lon"] = geo_df["center"].apply(lambda p: p.x)
geo_df["lat"] = geo_df["center"].apply(lambda p: p.y)

# 添加散点图层以固定显示文本
fig.add_trace(
    go.Scattergeo(
        lon=geo_df["lon"],
        lat=geo_df["lat"],
        text=geo_df["text"],
        mode="text",
        textfont=dict(size=6, color="darkred"),  # 设置字体大小，您可以根据需要调整
    )
)

fig.update_geos(fitbounds="locations", visible=False)

fig.update_layout(
    title_text="Model2-Rotation Benefit of Kharif",
    width=1800,  # 宽度, 您可以根据需要调整
    height=1200,  # 高度, 您可以根据需要调整
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(width=1800, height=1200, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()
fig.write_image("#model2_rotate_Kharif.png", scale=2)

C:\Users\redhawc\AppData\Local\Temp\ipykernel_55160\1907940586.py:25: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [ ]:
geo_df

,ST_NM,geometry,crop,state,Rabi,Summer,text,center,lon,lat
0,Arunachal Pradesh,"POLYGON ((95.23392 26.68246, 95.23282 26.70579...",wheat,Arunachal Pradesh,52920,0.0,"Area Size:<br>rice: 53.0k, <br> maize: 53.0k, ...",POINT (94.67483 28.03607),94.674833,28.036066
1,Assam,"POLYGON ((95.19465 27.03132, 95.15008 26.99934...",wheat,Assam,776459.25,0.0,"Area Size:<br>rice: 776.0k, <br> cotton: 776.0...",POINT (92.82634 26.35498),92.826343,26.354978
2,Chandigarh,"POLYGON ((76.83806 30.75487, 76.83301 30.73887...",wheat,Chandigarh,378,0.0,"Area Size:<br>rice: 0.0k, <br> maize: 0.0k",POINT (76.78078 30.72836),76.780780,30.728361
3,Karnataka,"POLYGON ((77.55144 18.29191, 77.57026 18.29252...",cotton,Karnataka,2283167.25,0.0,"Area Size:<br>rice: 1607.0k, <br> cotton: 2283...",POINT (76.16747 14.71041),76.167475,14.710406
4,Manipur,"POLYGON ((94.67545 25.44561, 94.67293 25.42398...",cotton,Manipur,67756.5,0.0,"Area Size:<br>rice: 68.0k, <br> maize: 68.0k, ...",POINT (93.87884 24.73508),93.878842,24.735076
5,Meghalaya,"POLYGON ((92.42522 25.02966, 92.40163 25.03363...",cotton,Meghalaya,45386.25,0.0,"Area Size:<br>rice: 64.0k, <br> cotton: 64.0k,...",POINT (91.27736 25.53558),91.277356,25.535577
6,Punjab,"POLYGON ((76.77673 30.90429, 76.78433 30.87791...",wheat,Punjab,1487430,0.0,"Area Size:<br>rice: 1487.0k, <br> cotton: 1487...",POINT (75.41580 30.84223),75.415804,30.842227
7,Rajasthan,"POLYGON ((74.52716 29.94279, 74.51002 29.90819...",wheat,Rajasthan,4668961.5,0.0,"Area Size:<br>rice: 4669.0k, <br> cotton: 4669...",POINT (73.84984 26.58441),73.849842,26.584408
8,Sikkim,"POLYGON ((88.74553 27.14203, 88.73457 27.13482...",wheat,Sikkim,26649,0.0,"Area Size:<br>rice: 27.0k, <br> maize: 27.0k, ...",POINT (88.47340 27.56992),88.473403,27.569923
9,Tripura,"POLYGON ((92.21170 24.50448, 92.22977 24.50244...",wheat,Tripura,89916.75,0.0,"Area Size:<br>rice: 90.0k, <br> cotton: 90.0k,...",POINT (91.73907 23.74514),91.739069,23.745138


In [ ]:
import plotly.graph_objects as go
import geopandas as gpd

# ...（读取地理数据的代码）
geojson_url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
geo_df = gpd.read_file(geojson_url)
# 假设 BenifitModel2 是您的数据集
geo_df = geo_df.merge(merged_df_Rabi, left_on="ST_NM", right_on="state")

# 创建 Choropleth 图层
fig = go.Figure(
    data=go.Choropleth(
        geojson=geojson_url,
        featureidkey="properties.ST_NM",
        locations=geo_df["state"],
        z=geo_df["Rabi"],  # 使用 'Rabi' 作为区分的依据
        text=geo_df["text"],  # 使用合并的 'text' 列
        colorscale=["lightgrey", "green"],  # 使用浅灰色到绿色的色阶
        hoverinfo="location+text",
        marker_line_color="burlywood",
    )
)

# 计算每个州的中心点坐标
geo_df["center"] = geo_df["geometry"].centroid
geo_df["lon"] = geo_df["center"].apply(lambda p: p.x)
geo_df["lat"] = geo_df["center"].apply(lambda p: p.y)

# 添加散点图层以固定显示文本
fig.add_trace(
    go.Scattergeo(
        lon=geo_df["lon"],
        lat=geo_df["lat"],
        text=geo_df["text"],
        mode="text",
        textfont=dict(size=6, color="darkred"),  # 设置字体大小，您可以根据需要调整
    )
)

fig.update_geos(fitbounds="locations", visible=False)

fig.update_layout(
    title_text="Model2-Rotation Benefit of Rabi",
    width=1800,  # 宽度, 您可以根据需要调整
    height=1200,  # 高度, 您可以根据需要调整
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(width=1800, height=1200, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()
fig.write_image("#model2_rotate_Rabi.png", scale=2)

C:\Users\redhawc\AppData\Local\Temp\ipykernel_55160\2598680620.py:25: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [ ]:
geo_df

,ST_NM,geometry,crop,state,Rabi,Summer,text,center,lon,lat
0,Arunachal Pradesh,"POLYGON ((95.23392 26.68246, 95.23282 26.70579...",wheat,Arunachal Pradesh,52920,NaN,Rotation Benifit:<br>wheat: 52920<br>Area Size...,POINT (94.67483 28.03607),94.674833,28.036066
1,Assam,"POLYGON ((95.19465 27.03132, 95.15008 26.99934...",wheat,Assam,776459.25,NaN,Rotation Benifit:<br>wheat: 776459.25<br>Area ...,POINT (92.82634 26.35498),92.826343,26.354978
2,Chandigarh,"POLYGON ((76.83806 30.75487, 76.83301 30.73887...",wheat,Chandigarh,378,NaN,Rotation Benifit:<br>wheat: 378<br>Area Size:<...,POINT (76.78078 30.72836),76.780780,30.728361
3,Karnataka,"POLYGON ((77.55144 18.29191, 77.57026 18.29252...",cotton,Karnataka,2283167.25,NaN,Rotation Benifit:<br>cotton: 2283167.25<br>Are...,POINT (76.16747 14.71041),76.167475,14.710406
4,Manipur,"POLYGON ((94.67545 25.44561, 94.67293 25.42398...",cotton,Manipur,67756.5,NaN,Rotation Benifit:<br>cotton: 67756.5<br>Area S...,POINT (93.87884 24.73508),93.878842,24.735076
5,Meghalaya,"POLYGON ((92.42522 25.02966, 92.40163 25.03363...",cotton,Meghalaya,45386.25,NaN,Rotation Benifit:<br>cotton: 45386.25<br>Area ...,POINT (91.27736 25.53558),91.277356,25.535577
6,Punjab,"POLYGON ((76.77673 30.90429, 76.78433 30.87791...",wheat,Punjab,1487430,NaN,Rotation Benifit:<br>wheat: 1487430<br>Area Si...,POINT (75.41580 30.84223),75.415804,30.842227
7,Rajasthan,"POLYGON ((74.52716 29.94279, 74.51002 29.90819...",wheat,Rajasthan,4668961.5,NaN,Rotation Benifit:<br>wheat: 4668961.5<br>Area ...,POINT (73.84984 26.58441),73.849842,26.584408
8,Sikkim,"POLYGON ((88.74553 27.14203, 88.73457 27.13482...",wheat,Sikkim,26649,NaN,Rotation Benifit:<br>wheat: 26649<br>Area Size...,POINT (88.47340 27.56992),88.473403,27.569923
9,Tripura,"POLYGON ((92.21170 24.50448, 92.22977 24.50244...",wheat,Tripura,89916.75,NaN,Rotation Benifit:<br>wheat: 89916.75<br>Area S...,POINT (91.73907 23.74514),91.739069,23.745138
